# Some utilities to check my dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import datetime
import sqlalchemy

In [ ]:
with open("../localconfig.json", "r") as f:
	config = json.load(f)

# Load entire dataset.

In [ ]:
connectionstr = f"mysql+pymysql://{config['dbuser']}:{config['dbpass']}@{config['dbhost']}/{config['dbname']}?charset=utf8mb4"
engine = sqlalchemy.create_engine(connectionstr)
engine.connect()

#solardf = pd.read_sql_table('production', engine)
sql =  '''
SELECT datetime, consumption, production FROM production 
	ORDER by datetime asc
'''
solardf = pd.read_sql(sql, engine)

In [ ]:
solardf

# some basic data checks.

In [ ]:
solardf['consumption'].describe()

In [ ]:
solardf['production'].describe()

In [ ]:
cons = (solardf['consumption'].min(), solardf['consumption'].max())
prod = (solardf['production'].min(), solardf['production'].max())
print (f"consumption min {cons[0]} max {cons[1]}")
print (f"production min {prod[0]} max {prod[1]}")

# Plots which let us visualize the distribution.

In [ ]:
solardf['production'].hist()

In [ ]:
solardf['consumption'].hist()

# Check for gaps in the data series.

Since I log every 10 seconds, look for gaps greater than 60 seconds

In [ ]:
# Calculate time interval from previous row.  
#data_intervals = solardf['datetime'].subtract(solardf['datetime'].shift(1)).dropna()
data_intervals = solardf['datetime'].diff().dropna()

# Add two columns:  interval as timedelta and as seconds
solardf['interval'] = data_intervals
solardf['intervalsec'] = data_intervals.apply(lambda x: x.seconds)

In [ ]:
longdelayds = solardf[solardf['intervalsec'] > 60]
longdelayds

In [ ]:
d = solardf[(solardf['datetime'] > datetime.datetime(2022, 5, 18, 12, 00, 00)) & (solardf['datetime'] < datetime.datetime(2022, 5, 18, 18, 15, 00))]
d.to_csv("dropped_data.csv")